# Imports

In [2]:
from numpy.random import seed
seed(888)
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.compat.v1 import set_random_seed
set_random_seed(404)

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import os
import numpy as np

from time import strftime
from PIL import Image

# Constants

In [4]:
X_TRAIN_PATH = "MNIST/digit_xtrain.csv"
X_TEST_PATH = "MNIST/digit_xtest.csv"
Y_TRAIN_PATH = "MNIST/digit_ytrain.csv"
Y_TEST_PATH = "MNIST/digit_ytest.csv"
TEST_IMG_PATH = "MNIST/test_img.png"

LOGGING_PATH = "tensorboard_mnsit_digit_logs/"

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMG_WIDTH = 28
IMG_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMG_HEIGHT*IMG_WIDTH*CHANNELS

# Gather Data

In [5]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=",", dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=",", dtype=int)

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=",", dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=",", dtype=int)

# Explore the Data

In [6]:
x_train_all.shape

(60000, 784)

In [7]:
y_train_all.shape

(60000,)

In [8]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

### Convert target values to one-hot encoding

In [9]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [10]:
y_train_all.shape

(60000, 10)

In [11]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Create Validation dataset from training data

In [12]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [13]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Setup Tensorflow Graph

In [14]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name="X")
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name="labels")

## Neurel Network Architecture
### Hyperparameters

In [15]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [16]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim,
                                     stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name="W")

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name="b2")

        layer_in = tf.matmul(input, w) + b
        
        if name=="out":
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
            
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        
        return layer_out

In [17]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
                      bias_dim=[n_hidden1],
                      name="layer_1")

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name="dropout_layer")

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2],
                      bias_dim=[n_hidden2],
                      name="layer_2")

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
                      bias_dim=[NR_CLASSES],
                      name="out")

model_name = f"{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}"

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [18]:
folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print("Successfully created directories!")

Successfully created directories!


# Loss, Optimisation & Metrics

### Defining Loss Function

In [19]:
with tf.name_scope("loss_calc"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

### Defining Optimizer

In [20]:
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

### Accuracy Metric

In [21]:
with tf.name_scope("accuracy_calc"):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [22]:
with tf.name_scope("performance"):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar("cost", loss)

# Run Session

In [23]:
sess = tf.Session()

### Setup Filewriter and Merge Summaries

In [24]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + "/train")
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + "/validation")

### Init all the variables

In [25]:
init = tf.global_variables_initializer()
sess.run(init)

## Batching the Data

In [26]:
size_of_batch = 1000

num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [27]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

## Training loop

In [28]:
for epoch in range(nr_epochs):
    
    # ============== Training Dataset ============== #
    for i in range(nr_iterations):
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train,
                                     labels=y_train)
        feed_dictinoray = {
            X:batch_x,
            Y:batch_y
        }
        sess.run(train_step, feed_dict=feed_dictinoray)
        
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictinoray)
    
    train_writer.add_summary(s, epoch)
    
    print(f"Epoch {epoch} \t| Training Accuracy = {batch_accuracy}")
    
    # ============== Validation Dataset ============== #
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)
    

print("Done Training")

Epoch 0 	| Training Accuracy = 0.5049999952316284
Epoch 1 	| Training Accuracy = 0.5989999771118164
Epoch 2 	| Training Accuracy = 0.7200000286102295
Epoch 3 	| Training Accuracy = 0.7419999837875366
Epoch 4 	| Training Accuracy = 0.7379999756813049
Epoch 5 	| Training Accuracy = 0.7419999837875366
Epoch 6 	| Training Accuracy = 0.7419999837875366
Epoch 7 	| Training Accuracy = 0.7599999904632568
Epoch 8 	| Training Accuracy = 0.7440000176429749
Epoch 9 	| Training Accuracy = 0.7580000162124634
Epoch 10 	| Training Accuracy = 0.7680000066757202
Epoch 11 	| Training Accuracy = 0.7599999904632568
Epoch 12 	| Training Accuracy = 0.746999979019165
Epoch 13 	| Training Accuracy = 0.7599999904632568
Epoch 14 	| Training Accuracy = 0.7599999904632568
Epoch 15 	| Training Accuracy = 0.7519999742507935
Epoch 16 	| Training Accuracy = 0.7440000176429749
Epoch 17 	| Training Accuracy = 0.7609999775886536
Epoch 18 	| Training Accuracy = 0.7630000114440918
Epoch 19 	| Training Accuracy = 0.75599998

# Make Prediction

In [29]:
img = Image.open(TEST_IMG_PATH)
img

In [30]:
# Because the training set images are black on white.
# But the test image is white on black.
bw = img.convert("L") 
img_array = np.invert(bw)
img_array.shape

(28, 28)

In [31]:
test_img = img_array.ravel()
test_img.shape

(784,)

In [32]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X: [test_img]})

In [36]:
print(f"Prediction for test images is {prediction}")
img

Prediction for test images is [2]


# Testing and Evaluation

In [34]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f"Accuracy on test set is {test_accuracy:0.2%}")

Accuracy on test set is 77.08%


# Reset for the Next Run

In [35]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()